In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
import os
import SolarStormwatchIIAnalysis as ssw
import hi_processing as hip
import sunpy.map as smap
import astropy.units as u
import matplotlib as mpl
import tables
from sklearn.neighbors import KernelDensity
from skimage.morphology import medial_axis
%matplotlib inline

C:\Users\Luke\Anaconda2\lib\site-packages\glymur\lib\config.py:151: UserWarning: Neither the openjp2 nor the openjpeg library could be loaded.  
  warnings.warn(msg)


In [ ]:
project_dirs = ssw.get_project_dirs()
ssw_out_name = os.path.join(project_dirs['out_data'], 'all_classifications_matched_ssw_events.hdf5')

# Load in the event tree dictionary to easily iterate over the events/craft/img type
ssw_event_tree = ssw.get_event_subject_tree(active=True)

# get handle to the ssw out data
ssw_out = tables.open_file(ssw_out_name, mode="r")

x = np.arange(0,1024)
y = np.arange(0, 1024)
xm, ym = np.meshgrid(x, y)
all_coords = np.vstack([xm.ravel(), ym.ravel()]).T

# Loop over events and craft
for event_k, event in ssw_event_tree.iteritems():

    if event_k != "ssw_009":
        continue

    for craft_k, craft in event.iteritems():

        # Get path to the normal and diff img types of this event, pull out the group of times
        norm_path = "/".join(['', event_k, craft_k, 'norm'])
        norm_times = ssw_out.get_node(norm_path)

        diff_path = "/".join(['', event_k, craft_k, 'diff'])
        diff_times = ssw_out.get_node(diff_path)

        # Set up the normalisation and colormap for the plots
        norm = mpl.colors.Normalize(vmin=0, vmax=len(diff_times._v_groups))
        cmap = mpl.cm.viridis
        for frame_count, (norm_t, diff_t) in enumerate(zip(norm_times, diff_times)):
            # Get the normal and diff coords
            norm_coords = pd.DataFrame.from_records(norm_t.pixel_coords.read())
            diff_coords = pd.DataFrame.from_records(diff_t.pixel_coords.read())
            xy = np.vstack([norm_coords['x'].values.ravel(), norm_coords['y'].values.ravel()]).T
            if norm_coords.shape[0] < 20:
                continue
                
            bandwidth = 50
            xy = np.vstack([norm_coords['x'].values.ravel(), norm_coords['y'].values.ravel()]).T
            ekde = KernelDensity(kernel='cosine', bandwidth=bandwidth).fit(xy)
            norm_epdf = np.reshape(ekde.score_samples(all_coords), xm.shape)
            
            xy = np.vstack([diff_coords['x'].values.ravel(), diff_coords['y'].values.ravel()]).T
            ekde = KernelDensity(kernel='cosine', bandwidth=bandwidth).fit(xy)
            diff_epdf = np.reshape(ekde.score_samples(all_coords), xm.shape)

            # Mark on the points
            fig, ax = plt.subplots(1, 2, figsize=(15, 15))
            
            # Pull out a binary blob at half the maximum denisty
            norm_binary = np.exp(norm_epdf.copy())
            m_thresh = np.median(norm_binary[np.nonzero(norm_binary)])
            p_thresh = np.max(norm_binary)/2.0
            print "m thresh:{}".format(m_thresh), "p thresh:{}".format(p_thresh)
            norm_binary[norm_binary < p_thresh] = 0
            norm_binary[norm_binary >= p_thresh] = 1
            skel = medial_axis(norm_binary)
            coords = np.nonzero(skel)
            
            ax[0].imshow(norm_binary, origin='lower', cmap=cmap)
            ax[0].plot(norm_coords['x'], norm_coords['y'], 'o', color='r', alpha=0.3)
            ax[0].plot(coords[1], coords[0], '-', color='r', linewidth=2)
            
            diff_binary = np.exp(diff_epdf.copy())
            m_thresh = np.median(diff_binary[np.nonzero(diff_binary)])
            p_thresh = np.max(diff_binary)/2.0
            print "m thresh:{}".format(m_thresh), "p thresh:{}".format(p_thresh)
            diff_binary[diff_binary < p_thresh] = 0
            diff_binary[diff_binary >= p_thresh] = 1
            skel = medial_axis(diff_binary)
            coords = np.nonzero(skel)
            
            coords = np.nonzero(skel)
            ax[1].imshow(diff_binary, origin='lower', cmap=cmap)
            ax[1].plot(diff_coords['x'], diff_coords['y'], 'o', color='r', alpha=0.3)
            ax[1].plot(coords[1], coords[0], '-', color='r', linewidth=2)
                
            # Label it up and format
            for a in ax.ravel():
                a.set_xlim(-100, 1123)
                a.set_ylim(-100, 1123)
                a.set_aspect('equal')
            
            plt.subplots_adjust(left=0.05,bottom=0.05,right=0.95,top=0.95)
            
            if frame_count==4:
                break
            
        break
        


In [ ]:
norm_coords = pd.DataFrame.from_records(norm_t.pixel_coords.read())
diff_coords = pd.DataFrame.from_records(diff_t.pixel_coords.read())
noise = np.random.randn(len(norm_coords))/10
xy = np.vstack([norm_coords['x'].values.ravel()+noise, norm_coords['y'].values.ravel()+noise]).T

for i,j in zip([0],[85]):
    norm_coords2 = norm_coords[i:j]
    xy = np.vstack([norm_coords2['x'].values.ravel(), norm_coords2['y'].values.ravel()]).T
    bandwidth = 40
    ekde = KernelDensity(kernel='cosine', bandwidth=bandwidth).fit(xy)
    norm_epdf = np.reshape(ekde.score_samples(all_coords), xm.shape)
    
    norm_binary = np.exp(norm_epdf.copy())           
    thresh = np.max(norm_binary)/3.0
    norm_binary[norm_binary < thresh] = 0
    norm_binary[norm_binary >= thresh] = 1
    
    fig, ax = plt.subplots(1, 2, figsize=(12, 12))         
    ax = ax.ravel()

    ax[0].imshow(norm_epdf, origin='lower', cmap=cmap)
    ax[1].imshow(norm_binary, origin='lower', cmap=cmap)
    #ax[2].imshow(bin0, origin='lower', cmap=cmap)
    #ax[3].imshow(bin1, origin='lower', cmap=cmap)
            
    # Label it up and format
    for a in ax:
        a.set_xlim(-100, 1123)
        a.set_ylim(-100, 1123)
        a.set_aspect('equal')

In [71]:
reload(ssw)

project_dirs = ssw.get_project_dirs()
    # Get the swpc cme database
swpc_cmes = ssw.load_swpc_events()

    # Import the SSW classifications data
ssw_out_name = os.path.join(project_dirs['out_data'], 'all_classifications_matched_ssw_events.hdf5')
ssw_out = tables.open_file(ssw_out_name, mode="r")

    # Iterate through the events, and annotate storm position on each plot.
for event in ssw_out.iter_nodes('/'):

        # Get ssw event number to loop up event HI start and stop time.
    ssw_num = int(event._v_name.split('_')[1])
    if ssw_num != 9:
        continue
        
    for craft in event:
        
        if craft._v_name != 'sta':
            continue
            
        if craft._v_name == 'sta':
            t_start = swpc_cmes.loc[ssw_num, 't_hi1a_start']
            t_stop = swpc_cmes.loc[ssw_num, 't_hi1a_stop']
        elif craft._v_name == 'stb':
            t_start = swpc_cmes.loc[ssw_num, 't_hi1b_start']
            t_stop = swpc_cmes.loc[ssw_num, 't_hi1b_stop']

        hi_files = hip.find_hi_files(t_start, t_stop, craft=craft._v_name, camera='hi1', background_type=1)

        files_c = hi_files[1:]
        files_p = hi_files[0:-1]

        for fc, fp in zip(files_c, files_p):

            for img_type in craft:

                if img_type._v_name == 'norm':
                        # Get Sunpy map of the image, convert to grayscale image with plain_normalise
                    hi_map = hip.get_image_plain(fc, star_suppress=False)
                elif img_type._v_name == 'diff':
                    hi_map = hip.get_image_diff(fc, fp, align=True, smoothing=True)

                # Get the time node for this file.
                time_label = hi_map.date.strftime('T%Y%m%d_%H%M%S')
                print time_label
                break
            break
        break
    break
        

T20121005_092901


In [70]:
img_type.